In [9]:
# Import fablib library
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

In [10]:
# Create a slice
slice = fablib.new_slice(name="dh-aes-exemplar")

In [11]:
# Define sites
sites = {
    'server': 'SALT',
    'switch': 'GPN',
    'clients': 'CLEM'
}

# Initialize dictionary to hold all nodes and their details
nodes = {}

# Add server
nodes['server'] = {
    'node': slice.add_node(name="server", site=sites['server'], cores=4, ram=8, disk=20, image='default_ubuntu_20'),
    'interfaces': []
}

# Add a client
nodes['station'] = {
    'node': slice.add_node(name="station", site=sites['clients'], cores=4, ram=8, disk=20, image='default_ubuntu_20'),
    'interfaces': []
}

# Add P4 switch
nodes['switch'] = {
    'node': slice.add_node(name="switch", site=sites['switch'], cores=32, ram=16, disk=40, image='default_ubuntu_20'),
    'interfaces': []
}

In [12]:
# Add network interfaces to each node
for name, node in nodes.items():
    num_interfaces = 2 if name == 'switch' else 1
    for i in range(num_interfaces):
        ifacename = 'net0' if name=='server' else ('net1' if name=='station' else f'net{i}')
        print(name, ifacename)
        interface = node['node'].add_component(name=ifacename, model='NIC_Basic').get_interfaces()[0]
        node['interfaces'].append(interface)

server net0
station net1
switch net0
switch net1


In [13]:
# Setup networks between clients and switch
networks = {}
networks['net0'] = slice.add_l2network(name='net0', interfaces=[nodes['server']['interfaces'][0], nodes['switch']['interfaces'][0]])
networks['net1'] = slice.add_l2network(name='net1', interfaces=[nodes['station']['interfaces'][0], nodes['switch']['interfaces'][1]])

In [14]:
# Submit the slice
slice.submit()


Retry: 15, Time: 404 sec


ID,d5f58e9a-fecf-48f8-8397-84bbc799490f
Name,dh-aes-exemplar
Lease Expiration (UTC),2024-04-23 04:37:50 +0000
Lease Start (UTC),2024-04-22 04:37:51 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
40250baf-713b-473f-993e-d42ad60bbdb2,server,4,8,100,default_ubuntu_20,qcow2,salt-w2.fabric-testbed.net,SALT,ubuntu,2001:400:a100:3010:f816:3eff:fe57:d761,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3010:f816:3eff:fe57:d761,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d17bbb80-35fa-4ce1-bc94-01bb8ea42848,station,4,8,100,default_ubuntu_20,qcow2,clem-w1.fabric-testbed.net,CLEM,ubuntu,2620:103:a006:12:f816:3eff:fea9:978c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fea9:978c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
b519881f-f1db-411a-b054-da8df0ec24e4,switch,32,16,100,default_ubuntu_20,qcow2,gpn-w5.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fea9:426a,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fea9:426a,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
9d513fc0-5968-45ad-8230-2180c4ebf766,net0,L2,L2STS,None,None,None,Active,
0f2415e9-d83c-4b79-bb56-faf7080c7f5b,net1,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
server-net0-p1,p1,server,net0,100,config,,02:10:E1:62:AA:17,enp7s0,enp7s0,None,4
station-net1-p1,p1,station,net1,100,config,,12:1B:80:25:AB:D0,enp6s0,enp6s0,None,6
switch-net0-p1,p1,switch,net0,100,config,,32:27:AF:52:E0:10,enp7s0,enp7s0,None,4
switch-net1-p1,p1,switch,net1,100,config,,2E:DF:DD:8D:66:11,enp6s0,enp6s0,None,4



Time to print interfaces 410 seconds


'd5f58e9a-fecf-48f8-8397-84bbc799490f'

In [2]:
import pandas as pd

slice = fablib.get_slice('dh-aes-exemplar')
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

,SSH command
Name,
server,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3010:f816:3eff:fe1d:1acb
station,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3010:f816:3eff:fe95:70c1
switch,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe3b:3f6f
